In [ ]:
import os
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import multiprocessing
import gzip
import string
import pandas as pd
import csv

In [ ]:
listing_titles = pd.read_csv('./dataset/Listing_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

In [22]:
sentences = [title.lower().translate(str.maketrans('', '', string.punctuation)) for title in listing_titles['Title'].to_list()]

In [23]:
sentences

['louis vuitton m40096 handbag priscilla multicolor canvas multicolor canvas',
 'louis vuitton petit noe drawstring shoulder bag monogram leather m42226 39sd442',
 'louis vuitton damier azur pochette bosphore shoulder bag n51112 lv auth yt523',
 'gucci bamboo 2way shoulder bag leather brown auth fm1002',
 'rank ab vintage gucci sherry line pvc leather clutch bag brown from japan a128',
 '1970s nyc bonnie cashin coach brown gray leather saddle pouch crossbody bag',
 'louis vuitton epi serviette fermoir business bag brown m54358 lv auth gt2071',
 'womens handbag',
 'chanel leo lion flap bag chevron lambskin medium',
 'gucci white  hobo  vintage handbag authenticated two tone  genuine leather ',
 'new fossil green plaid canvas estate cvs briefcase  cross body  handutility bag',
 'extra large colorful abstract print chenille carpet bag',
 'prada saffiano double zip tote',
 'chanel diamond cc double zip clutch with chain quilted lambskin',
 'valentino by mario valentino mandolino backpack w

In [24]:
def create_wordvecs(corpus, model_name):
    from gensim.models.word2vec import Word2Vec
    from gensim.models.phrases import Phrases, Phraser
    from collections import defaultdict
    
    print (len(corpus))
    

    phrases = Phrases(corpus, min_count=30, progress_per=10000)
    print ("Made Phrases")
    
    bigram = Phraser(phrases)
    print ("Made Bigrams")
    
    sentences = phrases[corpus]
    print ("Found sentences")
    word_freq = defaultdict(int)

    for sent in sentences:
        for i in sent:
            word_freq[i]+=1

    print (len(word_freq))
    
    print ("Training model now...")
    w2v_model = Word2Vec(min_count=1,
                        window=2,
                        sample=6e-5,
                        alpha=0.03,
                        min_alpha=0.0007,
                        negative=20,
                        workers=8)
    w2v_model.build_vocab(sentences, progress_per=10000)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
    os.makedirs("./trainset", exist_ok=True)
    w2v_model.wv.save_word2vec_format(f"trainset/{model_name}.txt")
create_wordvecs(sentences, "word_vecs")

19999921
Made Phrases
Made Bigrams
Found sentences
2334
Training model now...
